# Command line script

#### Command to bring up cluster (and sanity check)

    docker-compose up-d
    docker-compose ps
    docker ps -a
    
#### Command to create kafka topic -- assessments

    docker-compose exec kafka kafka-topics --create --topic assessments --partitions 1 --replication-factor 1 --if-not-exists --zookeeper zookeeper:32181
    
#### Command to check the kafka topic

    docker-compose exec kafka kafka-topics --describe --topic assessments  --zookeeper zookeeper:32181
    
#### Commands to publish and consume assessments
    (Publish)
    docker-compose exec mids bash -c "cat /w205/project-2-caseyhyoon/assessment-attempts-20180128-121051-nested.json | jq '.[]' -c | kafkacat -P -b kafka:29092 -t assessments"
    
    (Consume)
    docker-compose exec mids bash -c "kafkacat -C -b kafka:29092 -t assessments -o beginning -e"
    
#### Commands to shutdown cluster (and sanity check)

    docker-compose down
    docker-compose ps
    docker ps -a

# Business Questions

1. What are the top 5 most certified exams?

2. What are the hardest exams?

# Pyspark code

In [1]:
import json
from pyspark.sql import Row

## Q1 What are the top 5 most certified exams?

#### Create dataframe by subscribing kafka topic

In [2]:
assessments = spark.read.format("kafka").option("kafka.bootstrap.servers", "kafka:29092").option("subscribe","assessments").option("startingOffsets", "earliest").option("endingOffsets", "latest").load() 
assessments

DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]

#### Casting json data as strings

In [3]:
assessments.cache()
assessments = assessments.select(assessments.value.cast('string'))

#### Custom lambda function for certifications, unrolling json data, registering temporary tables

In [4]:
def my_lambda_certifications(x):
    
    raw_dict = json.loads(x.value)
    my_list = []
    certification = 0
    if "certification" in raw_dict:
        if raw_dict['certification']:
            certification += 1
    my_dict = {"exam_name": raw_dict['exam_name'],
               "certification": certification}
    my_list.append(Row(**my_dict))
    
    return my_list

certifications_assessments = assessments.rdd.flatMap(my_lambda_certifications).toDF()
certifications_assessments.registerTempTable('exam_certs')


#### SQL Query on temporary table

```sql
SELECT exam_name, SUM(certification) as sum_cert
FROM exam_certs
GROUP BY exam_name
ORDER BY sum_cert DESC
LIMIT 5
```

In [5]:
most_certified = spark.sql("SELECT exam_name, SUM(certification) as sum_cert FROM exam_certs GROUP BY exam_name ORDER BY sum_cert DESC LIMIT 5")
most_certified.show(truncate=False)


+-------------------------------+--------+
|exam_name                      |sum_cert|
+-------------------------------+--------+
|Learning Git                   |394     |
|Introduction to Python         |162     |
|Intermediate Python Programming|158     |
|Introduction to Java 8         |158     |
|Learning to Program with R     |128     |
+-------------------------------+--------+



ANSWER:
    1. Learning Git
    2. Introduction to Python
    3. Introduction to Java 8
    4. Intermediate Python Programming
    5. Learning to Program with R

#### Writing to HDFS

In [6]:
most_certified.write.parquet("/tmp/most_certified")

## Q2 What are the hardest exams?

#### Custom lambda function for difficulty, unrolling json data, registering temporary tables

In [7]:
def my_lambda_difficulty(x):
    
    raw_dict = json.loads(x.value)
    my_list = []
    
    if "sequences" in raw_dict:
        
        if "counts" in raw_dict["sequences"]:
            
            if "correct" in raw_dict["sequences"]["counts"] and "total" in raw_dict["sequences"]["counts"]:
                    
                my_dict = {"exam_name": raw_dict["exam_name"],
                           "correct": raw_dict["sequences"]["counts"]["correct"], 
                           "total": raw_dict["sequences"]["counts"]["total"]}
                my_list.append(Row(**my_dict))
    
    return my_list

In [8]:
difficulty_assessments = assessments.rdd.flatMap(my_lambda_difficulty).toDF()
difficulty_assessments.registerTempTable('difficulty')


#### SQL Query on temporary table

```sql
SELECT exam_name, avg((total-correct)/total) as incorrect_rating
FROM difficulty
GROUP BY exam_name
ORDER BY incorrect_rating DESC
LIMIT 5

```

In [9]:
difficulty = spark.sql("SELECT exam_name, avg((total-correct)/total) as incorrect_rating FROM difficulty GROUP BY exam_name ORDER BY incorrect_rating DESC limit 5")
difficulty.show(truncate=False)


+-------------------------------------------+------------------+
|exam_name                                  |incorrect_rating  |
+-------------------------------------------+------------------+
|Client-Side Data Storage for Web Developers|0.8               |
|Native Web Apps for Android                |0.75              |
|View Updating                              |0.75              |
|Arduino Prototyping Techniques             |0.6666666666666667|
|Mastering Advanced Git                     |0.6397058823529411|
+-------------------------------------------+------------------+



ANSWER:
    1. Client-Side Data Storage for Web Developers
    2. View Updating
    3. Native Web Apps for Android
    4. Arduino Prototyping Techniques
    5. Mastering Advanced 

#### Writing to HDFS

In [10]:
difficulty.write.parquet("/tmp/difficulty")